In [425]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import base

import warnings
warnings.filterwarnings('ignore')

In [426]:
#https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/

In [427]:
training_data = pd.read_csv("../data/TrainingSet.csv",index_col=0)
submission_labels = pd.read_csv("../data/SubmissionRows.csv", index_col=0)


In [428]:
#df = training_data.melt(id_vars=['Series Code', 'Country Name','Series Name'], var_name='Year', value_name='indicator',ignore_index = False)
df = training_data.melt(id_vars=['Series Code', 'Country Name'], var_name='Year', value_name='indicator',ignore_index = False, value_vars=['1972 [YR1972]','1973 [YR1973]','1974 [YR1974]','1975 [YR1975]',\
                                                                                                                                          '1976 [YR1976]','1977 [YR1977]','1978 [YR1978]','1979 [YR1979]', \
                                                                                                                                          '1980 [YR1980]',  '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]', \
                                                                                                                                          '1984 [YR1984]','1985 [YR1985]','1986 [YR1986]','1987 [YR1987]', \
                                                                                                                                          '1988 [YR1988]','1989 [YR1989]','1990 [YR1990]','1991 [YR1991]', \
                                                                                                                                          '1992 [YR1992]','1993 [YR1993]','1994 [YR1994]','1995 [YR1995]', \
                                                                                                                                          '1996 [YR1996]', '1997 [YR1997]','1998 [YR1998]','1999 [YR1999]', \
                                                                                                                                          '2000 [YR2000]','2001 [YR2001]','2002 [YR2002]','2003 [YR2003]' , \
                                                                                                                                          '2004 [YR2004]','2005 [YR2005]','2006 [YR2006]','2007 [YR2007]'])

df['Year'] = df['Year'].str.extract('(\d+)', expand=False) #.4stype(i4t),'2005 [YR2005]','2006 [YR2006]'

#df = df.sort_values(['Week', 'Product_Code'])

pd.set_option('display.max_colwidth', -1)

df['Year'] = pd.to_datetime(df['Year'], format='%Y')

df.fillna(0, inplace=True)

df.head()


,Series Code,Country Name,Year,indicator
0,allsi.bi_q1,Afghanistan,1972-01-01,0.0
1,allsp.bi_q1,Afghanistan,1972-01-01,0.0
2,allsa.bi_q1,Afghanistan,1972-01-01,0.0
4,allsi.gen_pop,Afghanistan,1972-01-01,0.0
5,allsp.gen_pop,Afghanistan,1972-01-01,0.0


In [429]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime
import re
import time
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
#import pydot_ng as pydot
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import plot_model
from keras.optimizers import SGD
from sklearn.model_selection import KFold, cross_val_score

In [439]:
# target Encoding Does not Work 
'''
target_encode_columns = ['Country Name','Series Code']
target = ['indicator']
target_encode_df = df[target_encode_columns + target] #.reset_index().drop(columns = 'index', axis = 1)
target_name = target[0]
target_df = pd.DataFrame()
for embed_col in target_encode_columns:
    val_map = target_encode_df.groupby(embed_col)[target].mean().to_dict()[target_name]
    target_df[embed_col] = target_encode_df[embed_col].map(val_map).values
    
score_target_drop = df.drop(target_encode_columns, axis = 1).reset_index().drop(columns = 'index', axis = 1)
score_target = pd.concat([score_target_drop, target_df], axis = 1)

score_target['Year']=score_target['Year'].dt.year
score_target.head()
'''


# Label Encoding 

encode_columns = ['Country Name','Series Code']
encode_df = df[encode_columns]
encode_df = encode_df.astype('str')
encode_df = encode_df.apply(LabelEncoder().fit_transform)
score_encode_drop = df.drop(encode_columns, axis = 1)
score_encode = pd.concat([score_encode_drop, encode_df], axis = 1)
score_encode['Year']=score_target['Year'].dt.year
score_target=score_encode


In [440]:
score_target.head()

,Year,indicator,Country Name,Series Code
0,1972,0.0,0,1296
1,1972,0.0,0,1299
2,1972,0.0,0,1293
4,1972,0.0,0,1298
5,1972,0.0,0,1301


In [441]:
X_train, X_test,y_train,y_test = train_test_split(score_target[['Year','Country Name','Series Code']],score_target['indicator'], test_size=0.7, random_state=42)


In [54]:
#https://www.kaggle.com/robikscube/tutorial-time-series-forecasting-with-xgboost
#not working

In [442]:
print(X_train.iloc[559])

Year            1979
Country Name    120 
Series Code     904 
Name: 161606, dtype: int64


In [ ]:

params = {'colsample_bytree': uniform(0.7, 0.3),
          'gamma': uniform(0, 0.5),
          'learning_rate': uniform(0.003, 0.1), # default 0.1 
          'max_depth': randint(2, 6), # default 3
          'n_estimators': randint(100, 250), # default 100
          'subsample': uniform(0.6, 0.4)}
xgb_model = xgb.XGBRegressor(objective="reg:linear",random_state=42)
time_split = TimeSeriesSplit(n_splits = 4)
#time_split = KFold(n_splits = 3)
#xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=2, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=42, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
x_model=xgb_search.fit(X_train,y_train)

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[18:42:02] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:42:08] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:42:20] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:42:39] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:43:04] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:43:23] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


Year            1.978000e+03
Country Name    3.089527e+10
Series Code     1.153301e+01
Name: 1220481, dtype: float64


In [351]:
y_pred = xgb_search.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print ('RMSE:', rms)

RMSE: 8859488903221.314


In [400]:
# connect predictions with outputs
for i in range(10):
	print(X_test.loc[i], "pred is: " + str(y_pred[i]))
    

Year            1.972000e+03
Country Name    1.346503e+09
Series Code     2.084733e-01
Name: 0, dtype: float64 pred is: 91806830000.0
Year            1.972000e+03
Country Name    1.346503e+09
Series Code     3.075955e-01
Name: 1, dtype: float64 pred is: -1857994100000.0
Year            1.972000e+03
Country Name    1.346503e+09
Series Code     6.558569e-01
Name: 2, dtype: float64 pred is: 91806830000.0
Year            1.972000e+03
Country Name    1.346503e+09
Series Code     8.910165e-01
Name: 3, dtype: float64 pred is: 139740050000.0


KeyError: 4

In [395]:
print(X_test.iloc[559],y_pred[559])

Year            1.993000e+03
Country Name    2.559124e+10
Series Code     1.906520e-01
Name: 4111780, dtype: float64 -2760975600.0


In [381]:
y_test['preds'] = y_hats

df_out = pd.merge(df,y_test[['preds']],how = 'left',left_index = True, right_index = True)

Int64Index([1077728, 1886092, 1383323, 1075330, 3073395, 5330552, 1073436,
            4741726, 3960318,  957585,
            ...
            1136074, 6197578, 1570006, 2234489, 4926484, 4304572, 1692743,
            6550634, 6423388, 6413414],
           dtype='int64', length=2051721)


In [413]:
submission_results=submission_labels.join(training_data)

submission_results=submission_results[['Country Name','Series Code']]

submission_results['Year']=2008

submission_results['indicator']=0

submission_results.head() 


,Country Name,Series Code,Year,indicator
559,Afghanistan,7.8,2008,0
618,Afghanistan,8.16,2008,0
753,Afghanistan,4.1,2008,0
1030,Afghanistan,6.1,2008,0
1896,Albania,7.8,2008,0


In [ ]:
# Target Encoding does not work 

In [420]:
target_encode_columns = ['Country Name','Series Code']
target = ['indicator']
target_encode_df = submission_results[target_encode_columns + target] #.reset_index().drop(columns = 'index', axis = 1)
target_name = target[0]
target_df = pd.DataFrame()
for embed_col in target_encode_columns:
    val_map = target_encode_df.groupby(embed_col)[target].mean().to_dict()[target_name]
    target_df[embed_col] = target_encode_df[embed_col].map(val_map).values
    
score_predict_drop = submission_results.drop(target_encode_columns, axis = 1).reset_index().drop(columns = 'index', axis = 1)
score_predict = pd.concat([score_target_drop, target_df], axis = 1)

score_predict['Year']= pd.to_datetime(score_target['Year'], format='%Y').dt.year




res_pred_01 = xgb_search.predict(score_predict[['Year', 'Country Name', 'Series Code']])

In [424]:
score_predict['predictions'] = res_pred_01

score_predict.head()


,Year,indicator,Country Name,Series Code,predictions
0,1972,0.0,0.0,0.0,9.180683e+10
1,1972,0.0,0.0,0.0,9.180683e+10
2,1972,0.0,0.0,0.0,9.180683e+10
3,1972,0.0,0.0,0.0,9.180683e+10
4,1972,0.0,0.0,0.0,9.180683e+10


In [418]:
submission_results=submission_labels.join(training_data)

submission_results=submission_results[['Country Name','Series Code']]

submission_results['Year']=2009

submission_results['indicator']=0

target_encode_columns = ['Country Name','Series Code']
target = ['indicator']
target_encode_df = submission_results[target_encode_columns + target] #.reset_index().drop(columns = 'index', axis = 1)
target_name = target[0]
target_df = pd.DataFrame()
for embed_col in target_encode_columns:
    val_map = target_encode_df.groupby(embed_col)[target].mean().to_dict()[target_name]
    target_df[embed_col] = target_encode_df[embed_col].map(val_map).values
    
score_predict_drop = submission_results.drop(target_encode_columns, axis = 1).reset_index().drop(columns = 'index', axis = 1)
score_predict = pd.concat([score_target_drop, target_df], axis = 1)

score_predict['Year']= pd.to_datetime(score_target['Year'], format='%Y').dt.year




res_pred = xgb_search.predict(score_predict[['Year', 'Country Name', 'Series Code']])